In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/dongmin/userdata/open-score-string-quartets


/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import sys
import subprocess
import warnings
from typing import Union, Any, Optional
import shutil
from pathlib import Path
from collections import Counter, defaultdict
from operator import itemgetter
from tempfile import NamedTemporaryFile, TemporaryDirectory

import math
import random

import json
import csv
import strictyaml as syaml

from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import numpy as np
import partitura as pt

from bs4 import BeautifulSoup
import requests

In [3]:
PathLike = Union[Path, str]

In [4]:
root_dir = Path(os.getcwd())
data_dir = root_dir / 'data'
score_dir = root_dir / 'scores'

In [5]:
data_dir, score_dir

(PosixPath('/home/dongmin/userdata/open-score-string-quartets/data'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores'))

## Load metadata

In [8]:
with open(data_dir / 'scores.yaml') as f:
  score_metadata = syaml.load(f.read())

score_metadata = score_metadata.data

mscore_data = [ dict( mscore_id=mscore_id, **obj ) for mscore_id, obj in score_metadata.items() ]

## convert MuseScore files to MusicXML

In [8]:
for d in tqdm(mscore_data):
  mscore_dir = score_dir / d['path'] 
  mscore_path = mscore_dir / f"sq{d['mscore_id']}.mscx"
  
  out_path = mscore_dir / f"sq{d['mscore_id']}.musicxml"
  
  cmd = ['musescore3', "-o", out_path, mscore_path, '-f']
    
  try:
    # convert MuseScore file to MusicXML
    ps = subprocess.run(
      cmd, 
      stdout=subprocess.DEVNULL, stderr=subprocess.PIPE
    )
    
    if ps.returncode != 0:
      raise Exception(
        "Command {} failed with code {}. MuseScore " "error messages:\n {}"
        .format(cmd, ps.returncode, ps.stderr.decode("UTF-8"))
      )
  
  except Exception as e:
    raise Exception(
      'Executing "{}" returned  {}.'.format(" ".join(cmd), e)
    )

 25%|██▍       | 30/122 [02:01<06:13,  4.06s/it]


KeyboardInterrupt: 

## Headless MuseScore Load + Render

In [15]:
def pt_load_mscx(
  filename:PathLike,
  mscore_exec:str='musescore3',
  validate:bool=False,
  force_note_ids:Union[bool,str]='keep',
):
  """
  modified function from partitura.io.musescore.load_via_musescore
  """
  if isinstance(filename, Path):
    filename = str(filename)
  
  with NamedTemporaryFile(suffix=".musicxml") as xml_fh:
    cmd = [mscore_exec, "-o", xml_fh.name, filename, "-f"]
    
    try:
      # convert MuseScore file to MusicXML
      ps = subprocess.run(
        cmd, 
        stdout=subprocess.DEVNULL, stderr=subprocess.PIPE
      )
      
      if ps.returncode != 0:
        raise Exception(
          "Command {} failed with code {}. MuseScore " "error messages:\n {}"
          .format(cmd, ps.returncode, ps.stderr.decode("UTF-8"))
        )
    
    except Exception as e:
      raise Exception(
        'Executing "{}" returned  {}.'.format(" ".join(cmd), e)
      )

    score = pt.load_musicxml(
      filename=xml_fh.name,
      validate=validate,
      force_note_ids=force_note_ids,
    )
  
  return score

In [16]:
def pt_render_musescore(
  score_data: pt.score.ScoreLike,
  fmt:str='png',
  out:Union[PathLike,None]=None,
  dpi:Optional[int]=90,
  mscore_exec:str='musescore3'
) -> Union[list[PathLike],list[np.array],None]:
  """
  modified function from partitura.io.musescore.render_musescore
  Render a score-like object using musescore.

  Parameters
  ----------
  score_data : ScoreLike
    Score-like object to be rendered
  fmt : {'png', 'pdf'}
    Output image format
  out : Path or str or None
    'png': OPTIONAL
    'pdf': REQUIRED
  dpi : int, optional
    Image resolution. 
    This option is ignored when `fmt` is 'pdf'. 
    Defaults to 90.

  Returns
  -------
  out : 
    1. list[PathLike]: list of paths to output images if out is provided
    2. list[np.array]: list of images if out is not provided
    3. None: if no image was generated
  """
  
  assert fmt in {'png', 'pdf'}, "Unsupported output format"
  
  if fmt == 'pdf':
    assert out is not None, "Output path is required for 'pdf' format"

  with TemporaryDirectory() as tmpdir:
    xml_fh = Path(tmpdir) / "score.musicxml"
    img_fh = Path(tmpdir) / f"score.{fmt}"

    pt.save_musicxml(score_data, xml_fh)

    cmd = [
      mscore_exec,
      # "-T",
      # "10",
      "-r",
      "{}".format(int(dpi)),
      "-o",
      os.fspath(img_fh),
      os.fspath(xml_fh),
      "-f",
    ]
    try:
      ps = subprocess.run(
        cmd, 
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
      )
      
      if ps.returncode != 0:
        raise Exception(
          "Command {} failed with code {}; stdout: {}; stderr: {}"
          .format(
            cmd,
            ps.returncode,
            ps.stdout.decode("UTF-8"),
            ps.stderr.decode("UTF-8"),
          )
        )
    
    except Exception as e:
      raise Exception(
        'Executing "{}" returned  {}.'
        .format(" ".join(cmd), e),
      )
    
    if fmt == "png":
      # gether all generated image files
      img_files = list(sorted(Path(tmpdir).glob(f"*.{fmt}")))
      
      # if no image was generated
      if len(img_files) < 1:
        return None
      
      # return images if out is not provided
      if out is None:
        out_images = [ cv2.imread(i_fp) for i_fp in img_files ]
        return out_images

      # return paths of images if out is provided
      else:
        out_files = [ out/i_fp.name for i_fp in img_files ]
        for i_fp, o_fp in zip(img_files, out_files):
          # make background white
          o_i = cv2.imread(i_fp, cv2.IMREAD_UNCHANGED)
          transparent_mask = o_i[:,:,3] == 0
          o_i[transparent_mask] = [255, 255, 255, 255]
          o_i = cv2.cvtColor(o_i, cv2.COLOR_BGRA2BGR)
          cv2.imwrite(o_fp, o_i)
        return out_files
    
    elif fmt == "pdf":
      if img_fh.is_file():
        shutil.copy(img_fh, out/img_fh.name)
      else:
        return None
    
    # if no image was generated
    return None

In [22]:
data = mscore_data[0]
mscore_dir = score_dir / data['path'] 
mscore_path = mscore_dir / f"sq{data['mscore_id']}.mscx"
musicxml_path = mscore_dir / f"sq{data['mscore_id']}.musicxml"

In [23]:
musicxml_path

PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/sq7313978.musicxml')

In [24]:
score = pt.load_musicxml(musicxml_path)

/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/io/importmusicxml.py:1014: UserWarning: Dashes end without dashes start
  warnings.warn("Dashes end without dashes start")
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/io/importmusicxml.py:927: UserWarning: Did not find a wedge start element for wedge stop!
  warnings.warn("Did not find a wedge start element for wedge stop!")
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/directions.py:514: UserWarning: error parsing "sul G" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/directions.py:514: UserWarning: error parsing "tranquillo" (UnexpectedCharacters)
  warnings.warn('error par

In [25]:
mscore_out_dir = mscore_dir / 'images' / 'original'
mscore_out_dir.mkdir(parents=True, exist_ok=True)

pt_render_musescore(
  score,
  fmt='png',
  out=mscore_out_dir,
  dpi=300,
)

[PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-01.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-02.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-03.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-04.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-05.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-06.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-07.png'),

## Render directly from MuseScore

In [7]:
def render_musescore(
  mscore_path:Union[PathLike,None],
  fmt:str='png',
  out:PathLike=None,
  dpi:Optional[int]=90,
  mscore_exec:str='musescore3'
) -> Union[list[PathLike],list[np.array],None]:
  """
  modified function from partitura.io.musescore.render_musescore
  Render a score-like object using musescore.

  Parameters
  ----------
  mscore_path : PathLike or None
    MuseScore file path to be rendered
  fmt : {'png', 'pdf'}
    Output image format
  out : Path or str or None
    'png': OPTIONAL
    'pdf': REQUIRED
  dpi : int, optional
    Image resolution. 
    This option is ignored when `fmt` is 'pdf'. 
    Defaults to 90.

  Returns
  -------
  out : 
    1. list[PathLike]: list of paths to output images if out is provided
    2. list[np.array]: list of images if out is not provided
    3. None: if no image was generated
  """
  
  assert fmt in {'png', 'pdf'}, "Unsupported output format"
  
  if fmt == 'pdf':
    assert out is not None, "Output path is required for 'pdf' format"

  with TemporaryDirectory() as tmpdir:
    img_fh = Path(tmpdir) / f"score.{fmt}"
    
    cmd = [
      mscore_exec,
      # "-T",
      # "10",
      "-r",
      "{}".format(int(dpi)),
      "-o",
      os.fspath(img_fh),
      os.fspath(mscore_path),
      "-f",
    ]
    try:
      ps = subprocess.run(
        cmd, 
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        env=dict(QT_QPA_PLATFORM='offscreen', **os.environ) # run MuseScore3 in headless mode
      )
      
      if ps.returncode != 0:
        raise Exception(
          "Command {} failed with code {}; stdout: {}; stderr: {}"
          .format(
            cmd,
            ps.returncode,
            ps.stdout.decode("UTF-8"),
            ps.stderr.decode("UTF-8"),
          )
        )
    
    except Exception as e:
      raise Exception(
        'Executing "{}" returned  {}.'
        .format(" ".join(cmd), e),
      )
    
    if fmt == "png":
      # gether all generated image files
      img_files = list(sorted(Path(tmpdir).glob(f"*.{fmt}")))
      
      # if no image was generated
      if len(img_files) < 1:
        return None
      
      # return images if out is not provided
      if out is None:
        out_images = [ cv2.imread(i_fp) for i_fp in img_files ]
        return out_images

      # return paths of images if out is provided
      else:
        out_files = [ out/i_fp.name for i_fp in img_files ]
        for i_fp, o_fp in zip(img_files, out_files):
          # make background white
          o_i = cv2.imread(i_fp, cv2.IMREAD_UNCHANGED)
          transparent_mask = o_i[:,:,3] == 0
          o_i[transparent_mask] = [255, 255, 255, 255]
          o_i = cv2.cvtColor(o_i, cv2.COLOR_BGRA2BGR)
          cv2.imwrite(o_fp, o_i)
        return out_files
    
    elif fmt == "pdf":
      if img_fh.is_file():
        shutil.copy(img_fh, out/img_fh.name)
      else:
        return None
    
    # if no image was generated
    return None

In [29]:
mscore_out_dir = mscore_dir / 'images' / 'original'
mscore_out_dir.mkdir(parents=True, exist_ok=True)

render_musescore(
  mscore_path,
  fmt='png',
  out=mscore_out_dir,
  dpi=300,
)

[PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-01.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-02.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-03.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-04.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-05.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-06.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/original/score-07.png'),

## Render with Lilypond

In [12]:
def render_lilypond(
  musicxml_path:PathLike,
  fmt:str="png",
  out:Optional[PathLike]=None,
) -> Union[list[PathLike],list[np.array],None]:
  """
  Render a score-like object using Lilypond

  Parameters
  ----------
  musicxml_path : PathLike
  fmt : {'png', 'pdf'}
    Output image format

  Returns
  -------
  out : 
    1. list[PathLike]: list of paths to output images if out is provided
    2. list[np.array]: list of images if out is not provided
    3. None: if no image was generated
  """
  assert fmt in {'png', 'pdf'}, "Unsupported output format"
  
  if fmt == 'pdf':
    assert out is not None, "Output path is required for 'pdf' format"

  with TemporaryDirectory() as tmpdir:
    pt_xml = Path(tmpdir) / "score.xml"
    pt.save_musicxml(pt.load_musicxml(musicxml_path), pt_xml)
    
    # convert musicxml to lilypond format (use stdout pipe)
    cmd1 = ["musicxml2ly", "-o-", str(pt_xml)]
    try:
      ps1 = subprocess.run(
        cmd1, stdout=subprocess.PIPE, check=False
      )
      if ps1.returncode != 0:
        raise Exception(
          "Command {} failed with code {}".format(cmd1, ps1.returncode)
        )
    
    except Exception as e:
      raise Exception(
        'Executing "{}" returned  {}.'
        .format(" ".join(cmd1), e),
      )

    # convert lilypond format (read from pipe of ps1) to image, and save to
    # temporary filename
    cmd2 = [
      "lilypond",
      "--{}".format(fmt),
      "-dprint-pages",
      "-o{}".format(tmpdir + '/score'),
      "-",
    ]
    try:
      ps2 = subprocess.run(cmd2, input=ps1.stdout, check=False)
      
      if ps2.returncode != 0:
        raise Exception(
          "Command {} failed with following error {}".format(cmd2, ps2.stderr)
        )
    
    except Exception as e:
      print(e)
      return
    
    if fmt == "png":
      # gether all generated image files
      img_files = list(sorted(Path(tmpdir).glob(f"*.{fmt}")))
      
      # if no image was generated
      if len(img_files) < 1:
        return None
      
      # return images if out is not provided
      if out is None:
        out_images = [ cv2.imread(i_fp) for i_fp in img_files ]
        return out_images

      # return paths of images if out is provided
      else:
        out_files = [ out/i_fp.name for i_fp in img_files ]
        for i_fp, o_fp in zip(img_files, out_files):
          shutil.copy(i_fp, o_fp)
        return out_files
    
    elif fmt == "pdf":
      pdf_file, *_ = list(Path(tmpdir).glob(f"*.{fmt}"))
      
      if pdf_file.is_file():
        shutil.copy(pdf_file, out/pdf_file.name)
      else:
        return None
    
    # if no image was generated
    return None

In [13]:
data = mscore_data[0]
mscore_dir = score_dir / data['path']
musicxml_path = mscore_dir / f"sq{data['mscore_id']}.musicxml"

lily_out_dir = mscore_dir / 'images' / 'lilypond'
lily_out_dir.mkdir(parents=True, exist_ok=True)

render_lilypond(
  musicxml_path,
  fmt='png',
  out=lily_out_dir,
)

/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/io/importmusicxml.py:1014: UserWarning: Dashes end without dashes start
  warnings.warn("Dashes end without dashes start")
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/io/importmusicxml.py:927: UserWarning: Did not find a wedge start element for wedge stop!
  warnings.warn("Did not find a wedge start element for wedge stop!")
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/directions.py:514: UserWarning: error parsing "sul G" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
/home/dongmin/.local/share/virtualenvs/open-score-string-quartets-wd2Cnojv/lib/python3.10/site-packages/partitura/directions.py:514: UserWarning: error parsing "tranquillo" (UnexpectedCharacters)
  warnings.warn('error par

[PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page1.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page10.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page11.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page12.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page13.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/lilypond/score-page14.png'),
 PosixPath('/home/dongmin/userdata/open-score-string-quartets/scores/Andrée,_Elfrida/String_Quartet_in_A_major/images/l